<h2>BOX OFFICE MOJO Scraper</h2>
<p>2018 Data</p>
Thomas Le

In [16]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np
from operator import truediv 

In [17]:
bo_data = 'box_office_data.csv'

In [18]:
#url = 'https://www.boxofficemojo.com/yearly/chart/?yr=' + str(year) + '&view=releasedate&view2=domestic&sort=gross&order=DESC&&p=.htm'

In [19]:
title = []
revenue = []
theaters = []
date = []

for year in range(2015,2019):
    url = 'https://www.boxofficemojo.com/yearly/chart/?yr=' + str(year) + '&view=releasedate&view2=domestic&sort=gross&order=DESC&&p=.htm'
    source = requests.get(url).text
    
    soup = BeautifulSoup(source, "html5lib")
    table = soup.find('td', colspan="3")
    core = table.tbody.tr
    
    for i in range(100):
        core = core.nextSibling
        title.append(core.a.text)
        revenue.append(core.td.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.text[1:])
        theaters.append(core.td.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.text)
        date.append(core.td.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.text + '/' + str(year)[2:])

        # clean titles for twitter search
        title = [i.replace('(2015)', '').replace('(2016)', '').replace('(2017)', '').replace('(2018)','').replace('II','2').replace('Vol. 2','2') for i in title]

        # revenue per theater calculations
        revenue = [i.replace(',', '') for i in revenue]
        theaters = [i.replace(',', '') for i in theaters]
        a = np.array(revenue)
        b = np.array(theaters)
        revenue_per_theater = (a.astype(np.float))/(b.astype(np.float))
        revenue_per_theater = revenue_per_theater.astype(np.int).tolist()

        # make dataframe
        biglist = [title] + [revenue] + [theaters] + [revenue_per_theater] + [date]
        biglist = np.array(biglist)
        biglist = np.transpose(biglist)
        df1 = pd.DataFrame(biglist, columns = ['Movie Title','Opening Weekend Revenue','Theaters Released In','Revenue Per Theater', 'Date Released'])

In [20]:
# remove all movie values with punctuation symbols like !:-, or '
tflist = df1['Movie Title'].str.contains("!|:|-|,|'", regex = True)
df1['tflist'] = tflist
df1 = df1[['Movie Title','tflist','Opening Weekend Revenue','Theaters Released In','Revenue Per Theater', 'Date Released']]
df1 = df1[df1['tflist'] == False] 
df1 = df1.drop(['tflist'], axis = 1)

# remove all movies that were released in less than 2000 theaters (easier to normalize revenue data later)
df1 = df1[df1['Theaters Released In'].astype(int) > 2000]

# remove movies whose entire titles are single common/generic words
# had inaccurate results when scraping for twitter with movie names identical to common/generic words like 'It' or 'Home'
# did a manual search of these (better method by using stopwords library and looping through?) If there were a lot more movies to scrape through this would be worth looking into 
df1 = df1.set_index('Movie Title')
common_movie_names = ['Home ','Spy','Sisters','Joy','Focus ','Max ','Pan','Aloha','Sing','It','Split','Life ','Tag','Alpha','Peppermint','Wonder','Vacation','The Man From U.N.C.L.E.']
#added a space after Home, Focus, Max, and Life (eg "home" to "home ") because they had a year or a number that was replaced with a space in the previous cell block
df1 = df1.drop(common_movie_names, axis = 0)
df1 = df1.reset_index()
display(df1)

# save to csv
df1.to_csv(bo_data, index = False)
    

,Movie Title,Opening Weekend Revenue,Theaters Released In,Revenue Per Theater,Date Released
0,Jurassic World,208806270,4274,48855,6/12/15
1,Inside Out,90440272,3946,22919,6/19/15
2,Furious 7,147187040,4004,36760,4/3/15
3,Minions,115718405,4301,26905,7/10/15
4,The Martian,54308575,3831,14176,10/2/15
5,Cinderella,67877361,3845,17653,3/13/15
6,Spectre,70403148,3929,17918,11/6/15
7,Pitch Perfect 2,69216890,3473,19930,5/15/15
8,Hotel Transylvania 2,48464322,3754,12910,9/25/15
9,Fifty Shades of Grey,85171450,3646,23360,2/13/15
